### Set the auth toke and site

In [1]:
const token = 'My auth token';
const site = 'da-dc';

async function logError(response) {
  console.error(`Request failed with status: ${response.status} ${response.statusText}`);
  
  console.error('Response Headers:');
  for (const [key, value] of response.headers.entries()) {
    console.error(`  ${key}: ${value}`);
  }
  
  try {
    const errorData = await response.json();
    console.error('Error Response Body:', errorData);
  } catch (e) {
    const errorText = await response.text();
    console.error('Error Response Body (text):', errorText);
  }
  
  throw new Error(`HTTP ${response.status}: ${response.statusText}`);
}

### Get the site config from EDS

In [ ]:
const response = await fetch(`https://admin.hlx.page/config/adobecom/sites/${site}.json`, {
  headers: {
    'x-auth-token': token,
  }
});

if (!response.ok) {
  logError(response);
}

const outputFile = `${site}.json`;
const siteCfg = await response.json();
await Deno.writeTextFile(outputFile, JSON.stringify(siteCfg, null, 2));
console.log(siteCfg);

### Update the site config

In [ ]:
delete siteCfg.created;
delete siteCfg.lastModified;
// reset the list of config admins
//siteCfg.access.admin.role.config_admin = [ "...@adobe.com", "...@adobe.com" ]
// add to the existing list of config admins
siteCfg.access.admin.role.config_admin.push("...@adobe.com");

console.log(siteCfg);

### Post the site config to EDS

In [ ]:
siteCfg.version = siteCfg.version + 1;

const responsUpdated = await fetch(`https://admin.hlx.page/config/adobecom/sites/${site}.json`, {
  method: 'POST',
  headers: {
    'content-type': 'application/json',
    'x-auth-token': token,
  },
  body: JSON.stringify(siteCfg)
});

if (!responsUpdated.ok) {
  logError(responsUpdated);
}

const outputFileUpdated = `${site}_updated.json`;
const siteCfUpdated = await responsUpdated.json();
await Deno.writeTextFile(outputFileUpdated, JSON.stringify(siteCfUpdated, null, 2));
console.log(siteCfUpdated);